# U4 - NMAIA Camera Test Tool
Author: George Gorospe, george.gorospe@nmaia.net (created 12/28/2025)

### This notebook is used to test the new and improved camera system for use in 2026 versions of the NMAIA AI/ML Software. This type of test tool is useful for testing the functions of the racer and it's software.

### There are three tools in this notebook:
Tool #1: Single image capture - shows how to use CV2 for image capture

Tool #2: Early example of TraitletCamera class, used for observing the camera and updating the image display widget.

Tool #3: Use of a widget button to save an image on button press.

In [ ]:
# Importing required libraries
#from jetcam.utils import bgr8_to_jpeg

import ipywidgets
from IPython.display import display
import cv2
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
## Capture and save a single image
#%matplotlib ipympl


# 1. Initialize the camera
# '0' refers to the default camera (usually the built-in webcam)
cap = cv2.VideoCapture(2)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video source.")
else:
    # 2. Capture a single frame
    ret, frame = cap.read() # ret is a boolean, frame is the image (numpy array)

    if ret:
        # 3. Convert the color space from BGR to RGB
        # OpenCV reads images in BGR, but Matplotlib expects RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # 4. Display the image using Matplotlib
        plt.imshow(frame_rgb)
        plt.title('Captured Image')
        plt.xticks([]) # Hide x-axis ticks
        plt.yticks([]) # Hide y-axis ticks
        plt.show()
        
        # Optional: Save the captured image to a file
        # cv2.imwrite("captured_image.png", frame)
    else:
        print("Failed to capture image.")

# 5. Release the camera
cap.release()


In [ ]:
### TraitletCamera Class ###
## This is a custom class implementing the TraitletC

# Import OpenCV for computer vision tasks (reading from the camera).
import cv2

# Import threading to allow the camera to run in the background
# without freezing the main program (Jupyter Notebook).
import threading

# Import traitlets to create "observable" variables.
# This allows us to trigger events whenever the image updates.
import traitlets

# Import Configurable so our class can interact nicely with Jupyter's configuration system.
from traitlets.config.configurable import Configurable

# Import NumPy because OpenCV images are just matrices of numbers.
import numpy as np


class TraitletCamera(Configurable):
    """
    A camera class that runs in a separate thread and updates a Traitlet
    whenever a new frame is captured.
    """

    # Define a 'trait'. This is a special type of variable.
    # Unlike a normal python variable, other parts of the code can "watch" this.
    # 'value' will hold the current video frame (as a numpy array).
    # traitlets.Any() means it can hold any data type (though we will use numpy arrays).
    value = traitlets.Any()
    
    def __init__(self, src=0, width=224, height=224, fps=30):
        """
        Constructor method to initialize the camera object.
        
        Args:
            src (int): The device index (e.g., 0 for /dev/video0, 2 for /dev/video2).
            width (int): The width to resize the image to.
            height (int): The height to resize the image to.
            fps (int): The target frames per second (informational, mainly).
        """
        # Initialize the parent class (Configurable)
        super().__init__()
        
        # Store the configuration parameters in the object
        self.src = src
        self.width = width
        self.height = height
        self.fps = fps
        
        # Create a flag to track if the camera loop is running or stopped
        self.running = False
        
        # This variable will hold the actual background thread object later
        self.thread = None
        
        # Initialize the OpenCV video capture object with the device index
        # This creates the connection to the physical hardware.
        self.cap = cv2.VideoCapture(self.src)
        
        # Check if the hardware connection was successful
        if not self.cap.isOpened():
            # If not, raise an error immediately so the user knows something is wrong.
            raise RuntimeError(f"Could not open video device /dev/video{src}")

    def start(self):
        """
        Starts the background thread that captures frames.
        """
        # If the camera is already running, do nothing (prevent duplicate threads).
        if self.running:
            return

        # Set the flag to True so the loop knows to keep going.
        self.running = True
        
        # Create the thread.
        # target=self._capture_loop: The function the thread will run.
        # daemon=True: Ensures the thread dies automatically if the main program closes.
        self.thread = threading.Thread(target=self._capture_loop, daemon=True)
        
        # Actually start the thread execution.
        self.thread.start()

    def stop(self):
        """
        Stops the background thread and releases the camera hardware.
        """
        # Set the flag to False. The 'while' loop in _capture_loop will see this and stop.
        self.running = False
        
        # If the thread exists, wait for it to finish its current loop before moving on.
        # This prevents race conditions where we release the camera while it's still reading.
        if self.thread:
            self.thread.join()
            
        # Release the hardware resource back to the OS.
        self.cap.release()

    def _capture_loop(self):
        """
        The main loop that runs inside the background thread.
        It continuously reads frames and updates the 'value' trait.
        """
        # Keep looping as long as the 'start()' method set this to True.
        while self.running:
            # Read a single frame from the camera.
            # ret: Boolean (True if read was successful, False if not).
            # frame: The image data (NumPy array).
            ret, frame = self.cap.read()
            
            # Only proceed if we actually got a valid frame.
            if ret:
                # Resize the image to the specified width and height.
                # This reduces data size and speeds up processing/display.
                frame = cv2.resize(frame, (self.width, self.height))
                
                # *** CRITICAL STEP ***
                # Update the 'value' trait.
                # This triggers any functions 'observing' this camera 
                # and updates any widgets 'linked' to this camera.
                self.value = frame
            
            # (Optional) We could add a time.sleep() here to limit FPS, 
            # but usually, the hardware read time limits it naturally.

# ==============================================================================
# Utility Function
# ==============================================================================

def bgr8_to_jpeg(value, quality=75):
    """
    Converts a raw OpenCV image (numpy array) into JPEG bytes.
    
    Why? ipywidgets.Image cannot display raw numpy arrays. 
    It needs a JPEG (or PNG) byte stream.
    
    Args:
        value: The numpy array (image).
        quality: JPEG compression quality (0-100).
        
    Returns:
        bytes: The image encoded as a JPEG string.
    """
    # If the image is None (camera hasn't started yet), return empty bytes.
    if value is None:
        return bytes()
        
    # use cv2.imencode to compress the raw matrix into a JPEG format.
    # It returns a tuple (success, encoded_image). We take index [1].
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
### Tool #2: displaying a live video feed
### Also includes a callback function example for image processing


# 1. Initialize the Camera
camera = TraitletCamera(src=2)
camera.start()

# This is an example of a callback function, 
# it runs each time a new frame is captured
def process_frame(change):
    new_image = change['new']

    brightness = np.mean(new_image)

# Start monitoring the camera, each time there is a new frame perform callback
camera.observe(process_frame, names='value')

# 2. Add the display
from IPython.display import display
import ipywidgets

# 3. Create the widget
image_widget = ipywidgets.Image(format='jpeg', width=camera.width,height=camera.height)

# 4. Link the widget and the camera feed, then display
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

In [ ]:
### Tool #3: Save an image on command
### Save an image from the feed on button-click ###

# Creating a button to save images from the stream to file.
# 1. Create the Button, Image Widget, and the Link
save_button = ipywidgets.Button(
    description='Save Snapshot',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to save the current frame',
    icon='camera' # (FontAwesome names without the `fa-` prefix)
)

# 2. Define the directory to save images
save_dir = 'snapshots'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 3. Define the function that runs when clicked
def save_snapshot(change):
    # Check if the camera actually has an image
    if camera.value is not None:
        # Create a unique filename using the current timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(save_dir, f"image_{timestamp}.jpg")
        
        # Save the image using OpenCV
        # camera.value is the raw numpy array (BGR format)
        cv2.imwrite(filename, camera.value)
        
        # Optional: Give feedback on the button itself
        save_button.description = f"Saved {timestamp}!"
        
        # Reset button text after 1 second (requires a separate thread or just leave it)
    else:
        save_button.description = "No Image!"

# 4. Link the button to the function
save_button.on_click(save_snapshot)

# 5. Display the Camera Widget AND the Button together
# We assume 'image_widget' was created in the previous step
layout = ipywidgets.VBox([image_widget, save_button])
display(layout)